# Code 2
Code intended to symbolically compute steps 1 (not yet 2)
## Defining coordinates, metrics, and variables/functions


In [1]:
# All packages needed: 
from sympy import *   #symbolic computations
import numpy as np    #usual numpy 
import time           #timer for computations

### Coordinates:

In [2]:
#Defining symbolic variable names
t, r, theta, phi = symbols('t, r, \\theta, \phi')
x ,y ,z = symbols('x, y, z')
#differentials
a, b, c, ds, dt, dr, dtheta, dphi = symbols('a, b, c, ds, dt, dr, d\\theta, d\phi')
dx ,dy ,dz = symbols('dx, dy, dz')
x_spherical=Matrix([t,r,theta,phi])
x_cartesian=Matrix([t,x,y,z])
dx_spherical=Matrix([dt,dr,dtheta,dphi])
dx_cartesian=Matrix([dt,dx,dy,dz])
xlist=[x_spherical, dx_spherical, x_cartesian, dx_cartesian]
# for i in range(0,4):
#     display(xlist[i])
###Replacing f at the end or anyway


#up-index to down index
def down(g,x):
    return (g*x).T

## Step :
### (a)

In [3]:
#Metric from the paper
#Paper Metric
Lambda=Function('\Lambda')(r,t)
N=Function('N')(r,t)
N_r=Function('N^r')(r,t)
R_r=Function('R')(r,t)
N=Function('N')(t)
ds2=-N**2 * dt**2 + Lambda**2 * (dr + N_r * dt) **2 + R_r **2 * (dtheta**2 + sin(theta)**2 * dphi**2)


#display line element
print('Line element:')
display(Eq(ds**2,ds2))
ds2=expand(ds2)

#compute metric
g_down=Matrix([[0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0]])
for i in range(0,4):
    for j in range(0,4):
        g_down[i,j]=ds2.coeff(xlist[1][i]*xlist[1][j])
        if str(xlist[1][i]) != str(xlist[1][j]):
            g_down[i,j]=g_down[i,j]/2
        
#display metric
print('Metric tensor:')
g_ab=symbols('g_{ab}')
g_up_ab=symbols('g^{ab}')
display(g_ab)
display(g_down)

display(g_up_ab)
g_up=simplify(g_down.inv())
display(g_up)


display(g_ab*g_up_ab)
simplify(g_down*g_down.inv())

Line element:


Eq(ds**2, -dt**2*N(t)**2 + (dr + dt*N^r(r, t))**2*\Lambda(r, t)**2 + (d\phi**2*sin(\theta)**2 + d\theta**2)*R(r, t)**2)

Metric tensor:


g_{ab}

Matrix([
[-N(t)**2 + N^r(r, t)**2*\Lambda(r, t)**2, N^r(r, t)*\Lambda(r, t)**2,          0,                         0],
[              N^r(r, t)*\Lambda(r, t)**2,           \Lambda(r, t)**2,          0,                         0],
[                                       0,                          0, R(r, t)**2,                         0],
[                                       0,                          0,          0, R(r, t)**2*sin(\theta)**2]])

g^{ab}

Matrix([
[       -1/N(t)**2,                          N^r(r, t)/N(t)**2,             0,                             0],
[N^r(r, t)/N(t)**2, \Lambda(r, t)**(-2) - N^r(r, t)**2/N(t)**2,             0,                             0],
[                0,                                          0, R(r, t)**(-2),                             0],
[                0,                                          0,             0, 1/(R(r, t)**2*sin(\theta)**2)]])

g^{ab}*g_{ab}

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

## Testing the code from steps 1 (a) to (h) with a simple metric

In [4]:
N=Function('N')(t)
X=Function('X')(t)
Y=Function('Y')(t)

g_down=Matrix([[-N**2, 0, 0, 0],[0, X**2, 0, 0],[0, 0, Y**2, 0],[0, 0, 0, sin(theta)**2 * Y**2]])
display(g_down)

#display metric
print('Metric tensor:')
g_ab=symbols('g_{ab}')
g_up_ab=symbols('g^{ab}')
display(g_ab)
display(g_down)

display(g_up_ab)
g_up=simplify(g_down.inv())
display(g_up)


display(g_ab*g_up_ab)
simplify(g_down*g_down.inv())


g_down

Matrix([
[-N(t)**2,       0,       0,                      0],
[       0, X(t)**2,       0,                      0],
[       0,       0, Y(t)**2,                      0],
[       0,       0,       0, Y(t)**2*sin(\theta)**2]])

Metric tensor:


g_{ab}

Matrix([
[-N(t)**2,       0,       0,                      0],
[       0, X(t)**2,       0,                      0],
[       0,       0, Y(t)**2,                      0],
[       0,       0,       0, Y(t)**2*sin(\theta)**2]])

g^{ab}

Matrix([
[-1/N(t)**2,          0,          0,                          0],
[         0, X(t)**(-2),          0,                          0],
[         0,          0, Y(t)**(-2),                          0],
[         0,          0,          0, 1/(Y(t)**2*sin(\theta)**2)]])

g^{ab}*g_{ab}

Matrix([
[-N(t)**2,       0,       0,                      0],
[       0, X(t)**2,       0,                      0],
[       0,       0, Y(t)**2,                      0],
[       0,       0,       0, Y(t)**2*sin(\theta)**2]])

In [5]:
g_down.det()

-N(t)**2*X(t)**2*Y(t)**4*sin(\theta)**2

### (b)

### $\Gamma^{\rho}_{\mu \nu} \to $ C[$\rho,\mu,\nu$]

In [6]:
#creation of an empty 4x4x4 array 
C=MutableDenseNDimArray(np.zeros((4,)*3))
C

#Computing the actual components and assigning them
for i in range(0,4):
    for j in range(0,4):
        for k in range(0,4):
            su=0
            for p in range(0,4):
                su=su+0.5*g_up[i,p]*(  diff(g_down[k,p],xlist[0][j]) + diff(g_down[p,j],xlist[0][k]) - diff(g_down[j,k],xlist[0][p])   )
            C[i,j,k]=su
C



[[[1.0*Derivative(N(t), t)/N(t), 0, 0, 0], [0, 1.0*X(t)*Derivative(X(t), t)/N(t)**2, 0, 0], [0, 0, 1.0*Y(t)*Derivative(Y(t), t)/N(t)**2, 0], [0, 0, 0, 1.0*Y(t)*sin(\theta)**2*Derivative(Y(t), t)/N(t)**2]], [[0, 1.0*Derivative(X(t), t)/X(t), 0, 0], [1.0*Derivative(X(t), t)/X(t), 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]], [[0, 0, 1.0*Derivative(Y(t), t)/Y(t), 0], [0, 0, 0, 0], [1.0*Derivative(Y(t), t)/Y(t), 0, 0, 0], [0, 0, 0, -1.0*sin(\theta)*cos(\theta)]], [[0, 0, 0, 1.0*Derivative(Y(t), t)/Y(t)], [0, 0, 0, 0], [0, 0, 0, 1.0*cos(\theta)/sin(\theta)], [1.0*Derivative(Y(t), t)/Y(t), 0, 1.0*cos(\theta)/sin(\theta), 0]]]

In [7]:
C[3,:,:]

[[0, 0, 0, 1.0*Derivative(Y(t), t)/Y(t)], [0, 0, 0, 0], [0, 0, 0, 1.0*cos(\theta)/sin(\theta)], [1.0*Derivative(Y(t), t)/Y(t), 0, 1.0*cos(\theta)/sin(\theta), 0]]

## Covariant Derivative
$\nabla_{b}A^{a}=\partial_{b}A^{a}+\Gamma_{bk}^{a}A^{k}=B^{a} \; _{b}$

and 

$\nabla_{b}A_{a}=\partial_{b}A_{a}-\Gamma_{ab}^{k}A_{k}=B_{ab}$

In [8]:
def covariant_derivative_up(C,xlist,x1):
    D=Matrix(np.zeros((4,)*2))
    for i in range(0,4):
        for j in range(0,4):
            tmp=0
            for k in range(0,4):
                tmp=tmp+C[j,i,k]*x1[k] #2
                D[i,j]=diff(x1[j],xlist[0][i])+tmp
    return D.T

def covariant_derivative_down(C,xlist,x1):
    D=Matrix(np.zeros((4,)*2))
    for i in range(0,4):
        for j in range(0,4):
            tmp=0
            for k in range(0,4):
                tmp=tmp+C[k,i,j]*x1[k] #2
                D[i,j]=diff(x1[j],xlist[0][i])-tmp
    return D.T

In [9]:
t1=Function('f')(t)
r1=Function('g')(r)
x1=Matrix([t1,r1,0,0])
display(x1)

x1_down=down(g_down,x1)
display(x1_down)

Matrix([
[f(t)],
[g(r)],
[   0],
[   0]])

Matrix([[-N(t)**2*f(t), X(t)**2*g(r), 0, 0]])

In [10]:
Bs1, Bs2, Bs3=symbols('B^{a}_{b}, B_{ab}, g_{ac}B^{c}_{b}')
B1=covariant_derivative_up(C,xlist,x1)
display(Bs1)
display(B1)

B2=covariant_derivative_down(C,xlist,x1_down)
display(Bs2)
display(simplify(B2))
display(Bs3 - Bs2)
display(simplify(g_down*B1-B2))

B^{a}_{b}

Matrix([
[Derivative(f(t), t) + 1.0*f(t)*Derivative(N(t), t)/N(t),               1.0*X(t)*g(r)*Derivative(X(t), t)/N(t)**2,                                 0,                                 0],
[                      1.0*g(r)*Derivative(X(t), t)/X(t), Derivative(g(r), r) + 1.0*f(t)*Derivative(X(t), t)/X(t),                                 0,                                 0],
[                                                      0,                                                       0, 1.0*f(t)*Derivative(Y(t), t)/Y(t),                                 0],
[                                                      0,                                                       0,                                 0, 1.0*f(t)*Derivative(Y(t), t)/Y(t)]])

B_{ab}

Matrix([
[-(N(t)*Derivative(f(t), t) + 1.0*f(t)*Derivative(N(t), t))*N(t),                             -1.0*X(t)*g(r)*Derivative(X(t), t),                                 0,                                                0],
[                              1.0*X(t)*g(r)*Derivative(X(t), t), (X(t)*Derivative(g(r), r) + 1.0*f(t)*Derivative(X(t), t))*X(t),                                 0,                                                0],
[                                                              0,                                                              0, 1.0*Y(t)*f(t)*Derivative(Y(t), t),                                                0],
[                                                              0,                                                              0,                                 0, 1.0*Y(t)*f(t)*sin(\theta)**2*Derivative(Y(t), t)]])

-B_{ab} + g_{ac}B^{c}_{b}

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

### (c)
Now we need $U^{\mu}$ such that:

$g_{\mu \nu}U^{\mu}U^{\nu}=-1$

And while we're at it, might as well:

$g_{\mu \nu} U_{\text{null}}^{\mu}U_{\text{null}}^{\nu}=0$

In [11]:
#displaying U 

ua=symbols('g_{ac}U^{c}U^{a}')
print('Then we have for U:')

u0=Function('u_0')(t)
u1=Function('u_1')(t)
print('Then we have for U:')
U=Matrix([u0,u1,0,0])

#contraction of U
display(U)
print('The contraction')
display(ua)
contraction=(down(g_down,U)*U)[0]
print('yields a value of:')
display(contraction)

#Choose normalization
normalization=-1
print('Choosing a normalization of',normalization, 'yields: ')
display(Eq(ua,normalization))

#computing u1(u0)
tmp=contraction-normalization
tmp1=solve(tmp,u1)
display(Eq(u1,tmp1[0]))

#Final U vector
arbitraty_u0=1/N
U=simplify(Matrix([u0,tmp1[0],0,0]).subs(u0,arbitraty_u0))
print('U is then given by:')
display(U)

Then we have for U:
Then we have for U:


Matrix([
[u_0(t)],
[u_1(t)],
[     0],
[     0]])

The contraction


g_{ac}U^{c}U^{a}

yields a value of:


...

Choosing a normalization of -1 yields: 


Eq(g_{ac}U^{c}U^{a}, -1)

Eq(u_1(t), -sqrt(N(t)**2*u_0(t)**2 - 1)/X(t))

U is then given by:


Matrix([
[1/N(t)],
[     0],
[     0],
[     0]])

### (d)
$h_{\mu \nu} = g_{\mu \nu} - U_{\mu} U_{\nu}$

In [12]:
#First, compute U_mu U_nu
U_down=down(g_down,U)
h=Matrix([[0, 0, 0, 0],[0,0, 0, 0],[0, 0, 0, 0],[0, 0, 0, 0]])
for i in range(0,4):
    for j in range(0,4):
        h[i,j]=g_down[i,j]-U_down[i]*U_down[j]
display(h)

Matrix([
[-2*N(t)**2,       0,       0,                      0],
[         0, X(t)**2,       0,                      0],
[         0,       0, Y(t)**2,                      0],
[         0,       0,       0, Y(t)**2*sin(\theta)**2]])

### (e)
$B_{\mu \nu} = \nabla_{\mu} U_{\nu}=\frac{\theta}{3}P_{\mu \nu} + \sigma_{\mu \nu} + \omega_{\mu \nu}$


In [13]:
B=simplify(covariant_derivative_down(C,xlist,U_down))
print('B:')
display(B)


B:


Matrix([
[0,                                 0,                                 0,                                                0],
[0, 1.0*X(t)*Derivative(X(t), t)/N(t),                                 0,                                                0],
[0,                                 0, 1.0*Y(t)*Derivative(Y(t), t)/N(t),                                                0],
[0,                                 0,                                 0, 1.0*Y(t)*sin(\theta)**2*Derivative(Y(t), t)/N(t)]])

### (f)
$\theta=P^{\mu \nu}B_{\mu \nu} = \nabla_{\mu} U^{\mu}=Tr(B) $

In [14]:
print('Expansion coefficient theta:')
theta_expansion=simplify(Trace(B*g_up))
display(theta_expansion)

Expansion coefficient theta:


2.0*Derivative(Y(t), t)/(N(t)*Y(t)) + 1.0*Derivative(X(t), t)/(N(t)*X(t))

### (g) 
$\sigma_{\mu \nu} = \frac{1}{2}(B_{\mu \nu}+ B_{\nu \mu})-\frac{1}{3}h_{\mu \nu} \theta$

and

$\sigma^2=\sigma_{\mu \nu}\sigma^{\mu \nu}= g^{\mu \alpha} g^{\nu \beta} \sigma_{\mu \nu} \sigma_{\alpha \beta}$

In [15]:
sigma=simplify(1/2*(B+B.T)-1/3*h*theta_expansion)
display(expand(simplify(sigma)))

Matrix([
[1.33333333333333*N(t)*Derivative(Y(t), t)/Y(t) + 0.666666666666667*N(t)*Derivative(X(t), t)/X(t),                                                                                                            0,                                                                                                           0,                                                                                                                                         0],
[                                                                                               0, -0.666666666666667*X(t)**2*Derivative(Y(t), t)/(N(t)*Y(t)) + 0.666666666666667*X(t)*Derivative(X(t), t)/N(t),                                                                                                           0,                                                                                                                                         0],
[                                                                          

In [16]:
simplify(1/2*(B+B.T))

Matrix([
[0,                                 0,                                 0,                                                0],
[0, 1.0*X(t)*Derivative(X(t), t)/N(t),                                 0,                                                0],
[0,                                 0, 1.0*Y(t)*Derivative(Y(t), t)/N(t),                                                0],
[0,                                 0,                                 0, 1.0*Y(t)*sin(\theta)**2*Derivative(Y(t), t)/N(t)]])

$\sigma^2=\sigma_{\mu \nu}\sigma^{\mu \nu}= g^{\mu \alpha} g^{\nu \beta} \sigma_{\mu \nu} \sigma_{\alpha \beta}$

In [17]:
#calculating sigma with upper indices
sigma_up=g_up*sigma*g_up

shear=0
for i in range(0,3):
    for j in range(0,3):
        shear+= sigma_up[i,j]*sigma[i,j]
print('shear \sigma^2 is given by: ')
display(expand(simplify(shear)))

shear \sigma^2 is given by: 


2.33333333333333*Derivative(Y(t), t)**2/(N(t)**2*Y(t)**2) + 0.666666666666666*Derivative(X(t), t)*Derivative(Y(t), t)/(N(t)**2*X(t)*Y(t)) + 1.0*Derivative(X(t), t)**2/(N(t)**2*X(t)**2)

### (h)
$\omega_{\mu \nu} = B_{[\mu \nu]}=\frac{1}{2}(B_{\mu \nu}-B_{\nu \mu})$

In [18]:
#computing the tensor
w=simplify(1/2*(B-B.T))
display(w)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

$\omega_{\mu \nu} + \omega_{\nu \mu}= B_{[\mu \nu]} + B_{[\nu \mu]}= B_{[\mu \nu]} - B_{[\mu \nu]} =0$

In [19]:
simplify(w+w.T)

Matrix([
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0],
[0, 0, 0, 0]])

In [20]:
w_up=g_up*w*g_up

vort=0
for i in range(0,3):
    for j in range(0,3):
        vort+= w_up[i,j]*w[i,j]
print('vorticity is given by:')
display(simplify(vort))


vorticity is given by:


0